In [3]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('ggplot')
%matplotlib inline

pd.set_option("display.max_columns", 30)
pd.set_option("display.max_colwidth", 100)
pd.set_option("display.precision", 3)

CSV_PATH = r"/Users/killer/work_station/PyLearn/Web/MLWeb/MLBlueprints/chapter2/magic.csv"

df = pd.read_csv(CSV_PATH)
df.columns

Index(['routablemask_link', 'routablemask_link/_text',
       'routablemask_link/_title', 'routablemask_link_numbers',
       'routablemask_content', 'imagebadge_value', 'imagebadge_value_numbers',
       'routable_link', 'routable_link/_text', 'routable_link/_title',
       'routable_link_numbers', 'listingtype_value', 'pricelarge_value',
       'pricelarge_value_prices', 'propertyinfo_value',
       'propertyinfo_value_numbers', 'fineprint_value',
       'fineprint_value_numbers', 'tozcount_number', 'tozfresh_value',
       'tablegrouped_values', 'tablegrouped_values_prices', '_PAGE_NUMBER'],
      dtype='object')

In [2]:
df.head().T

,0,1,2,3,4
routablemask_link,"http://www.zillow.com/b/2-Ellwood-St-New-York-NY/40.85899,-73.92814_ll/","http://www.zillow.com/b/603-W-140th-St-New-York-NY/40.82376,-73.95335_ll/",http://www.zillow.com/homedetails/9-E-129th-St-1-New-York-NY-10035/2100761096_zpid/,http://www.zillow.com/homedetails/710-Riverside-Dr-APT-2C-New-York-NY-10031/124451755_zpid/,http://www.zillow.com/homedetails/413-E-84th-St-APT-8-New-York-NY-10028/2100761260_zpid/
routablemask_link/_text,5 photos,10 photos,NaN,9 photos,5 photos
routablemask_link/_title,NaN,NaN,NaN,NaN,NaN
routablemask_link_numbers,5,10,NaN,9,5
routablemask_content,NaN,NaN,NaN,NaN,NaN
imagebadge_value,5 photos,10 photos,NaN,9 photos,5 photos
imagebadge_value_numbers,5,10,NaN,9,5
routable_link,"http://www.zillow.com/b/2-Ellwood-St-New-York-NY/40.85899,-73.92814_ll/","http://www.zillow.com/b/603-W-140th-St-New-York-NY/40.82376,-73.95335_ll/",http://www.zillow.com/homedetails/9-E-129th-St-1-New-York-NY-10035/2100761096_zpid/,http://www.zillow.com/homedetails/710-Riverside-Dr-APT-2C-New-York-NY-10031/124451755_zpid/,http://www.zillow.com/homedetails/413-E-84th-St-APT-8-New-York-NY-10028/2100761260_zpid/
routable_link/_text,2 Ellwood St,603 W 140th St,"9 E 129th St # 1, New York, NY10035","710 Riverside Dr APT 2C, New York, NY10031","413 E 84th St APT 8, New York, NY10028"
routable_link/_title,"2 Ellwood St APT 5H, New York, NY Real Estate","603 W 140th St APT 44, New York, NY Real Estate","9 E 129th St # 1, New York, NY Real Estate","710 Riverside Dr APT 2C, New York, NY Real Estate","413 E 84th St APT 8, New York, NY Real Estate"


In [5]:
# multiple units
mu = df[df['listingtype_value'].str.contains('Apartments For')]

# single units
su = df[df['listingtype_value'].str.contains('Apartment For')]

In [6]:
len(mu)

161

In [7]:
len(su)

339

In [8]:
su['propertyinfo_value']

2                                               1 bd • 1 ba
3                                 2 bds • 2 ba • 1,016 sqft
4                                             Studio • 1 ba
5                                              2 bds • 1 ba
7                                              2 bds • 1 ba
10                                              1 bd • 1 ba
11     1 bd • 1 ba • 496 sqft • 4,502 sqft lot • Built 1900
12                                            Studio • 1 ba
13                                              1 bd • 1 ba
17                                              1 bd • 1 ba
18                                            Studio • 1 ba
19                                              1 bd • 1 ba
20                                   1 bd • 1 ba • 700 sqft
21                                              1 bd • 1 ba
23                                 Studio • 1 ba • 487 sqft
24                                             2 bds • 1 ba
27                                   1 b

In [9]:
# 检查没有包含'bd'或'Studio'的行数
len(su[~su['propertyinfo_value'].str.contains('Studio') | su['propertyinfo_value'].str.contains('bd'))])

SyntaxError: invalid syntax (<ipython-input-9-2fa14217374f>, line 2)

In [10]:
# 检查没有包含'bd'或'Studio'的行数
len(su[~(su['propertyinfo_value'].str.contains('Studio') | su['propertyinfo_value'].str.contains('bd'))])

0

In [11]:
# 检查没有包含'ba'的行数
len(su[~(su['propertyinfo_value'].str.contains('ba'))])

6

In [12]:
# 选择拥有浴室的房源
# 首先选出没有浴室的房源
no_baths = su[~(su['propertyinfo_value'].str.contains('ba'))]

# 再排除那些缺失了浴室信息的房源
sucln = su[~su.index.isin(no_baths.index)]

In [13]:
# 继续解析卧室和浴室的信息，使用项目符号进行切分
def parse_info(row):
    if not 'sqft' in row:
        br, ba = row.split('')[:2]
        sqft = np.nan
    else:
        br, ba, sqft = row.split('.')[:3]
    return pd.Series({'Beds':br, 'Baths':ba, 'Sqft': sqft})

attr = sucln['propertyinfo_value'].apply(parse_info)

attr

ValueError: empty separator

In [14]:
# 继续解析卧室和浴室的信息，使用项目符号进行切分
def parse_info(row):
    if not 'sqft' in row:
        br, ba = row.split(' ')[:2]
        sqft = np.nan
    else:
        br, ba, sqft = row.split('.')[:3]
    return pd.Series({'Beds':br, 'Baths':ba, 'Sqft': sqft})

attr = sucln['propertyinfo_value'].apply(parse_info)

attr

ValueError: need more than 1 value to unpack

In [15]:
sucln

,routablemask_link,routablemask_link/_text,routablemask_link/_title,routablemask_link_numbers,routablemask_content,imagebadge_value,imagebadge_value_numbers,routable_link,routable_link/_text,routable_link/_title,routable_link_numbers,listingtype_value,pricelarge_value,pricelarge_value_prices,propertyinfo_value,propertyinfo_value_numbers,fineprint_value,fineprint_value_numbers,tozcount_number,tozfresh_value,tablegrouped_values,tablegrouped_values_prices,_PAGE_NUMBER
2,http://www.zillow.com/homedetails/9-E-129th-St-1-New-York-NY-10035/2100761096_zpid/,NaN,NaN,NaN,NaN,NaN,NaN,http://www.zillow.com/homedetails/9-E-129th-St-1-New-York-NY-10035/2100761096_zpid/,"9 E 129th St # 1, New York, NY10035","9 E 129th St # 1, New York, NY Real Estate",9; 129; 1,Apartment For Rent,"$1,750/mo",1750.0,1 bd • 1 ba,1; 1,NaN,NaN,48.0,minutes ago,NaN,NaN,1
3,http://www.zillow.com/homedetails/710-Riverside-Dr-APT-2C-New-York-NY-10031/124451755_zpid/,9 photos,NaN,9.0,NaN,9 photos,9.0,http://www.zillow.com/homedetails/710-Riverside-Dr-APT-2C-New-York-NY-10031/124451755_zpid/,"710 Riverside Dr APT 2C, New York, NY10031","710 Riverside Dr APT 2C, New York, NY Real Estate",710; 2,Apartment For Rent,"$3,000/mo",3000.0,"2 bds • 2 ba • 1,016 sqft",2; 2; 1016,NaN,NaN,1.0,hour ago,NaN,NaN,1
4,http://www.zillow.com/homedetails/413-E-84th-St-APT-8-New-York-NY-10028/2100761260_zpid/,5 photos,NaN,5.0,NaN,5 photos,5.0,http://www.zillow.com/homedetails/413-E-84th-St-APT-8-New-York-NY-10028/2100761260_zpid/,"413 E 84th St APT 8, New York, NY10028","413 E 84th St APT 8, New York, NY Real Estate",413; 84; 8,Apartment For Rent,"$2,300/mo",2300.0,Studio • 1 ba,1,NaN,NaN,2.0,hours ago,NaN,NaN,1
5,http://www.zillow.com/homedetails/324-Pleasant-Ave-APT-6E-New-York-NY-10035/2100761258_zpid/,7 photos,NaN,7.0,NaN,7 photos,7.0,http://www.zillow.com/homedetails/324-Pleasant-Ave-APT-6E-New-York-NY-10035/2100761258_zpid/,"324 Pleasant Ave APT 6E, New York, NY10035","324 Pleasant Ave APT 6E, New York, NY Real Estate",324; 6,Apartment For Rent,"$2,500/mo",2500.0,2 bds • 1 ba,2; 1,NaN,NaN,2.0,hours ago,NaN,NaN,1
7,http://www.zillow.com/homedetails/217-Thompson-St-APT-6-New-York-NY-10012/2100761371_zpid/,NaN,NaN,NaN,NaN,NaN,NaN,http://www.zillow.com/homedetails/217-Thompson-St-APT-6-New-York-NY-10012/2100761371_zpid/,"217 Thompson St APT 6, New York, NY10012","217 Thompson St APT 6, New York, NY Real Estate",217; 6,Apartment For Rent,"$2,800/mo",2800.0,2 bds • 1 ba,2; 1,NaN,NaN,3.0,hours ago,NaN,NaN,1
10,http://www.zillow.com/homedetails/340-W-47th-St-APT-4C-New-York-NY-10036/2108861613_zpid/,NaN,NaN,NaN,NaN,NaN,NaN,http://www.zillow.com/homedetails/340-W-47th-St-APT-4C-New-York-NY-10036/2108861613_zpid/,"340 W 47th St APT 4C, New York, NY10036","340 W 47th St APT 4C, New York, NY Real Estate",340; 47; 4,Apartment For Rent,"$2,490/mo",2490.0,1 bd • 1 ba,1; 1,NaN,NaN,6.0,hours ago,NaN,NaN,1
11,http://www.zillow.com/homedetails/1356-1st-Ave-APT-5B-New-York-NY-10021/31539199_zpid/,9 photos,NaN,9.0,NaN,9 photos,9.0,http://www.zillow.com/homedetails/1356-1st-Ave-APT-5B-New-York-NY-10021/31539199_zpid/,"1356 1st Ave APT 5B, New York, NY10021","1356 1st Ave APT 5B, New York, NY Real Estate",1356; 1; 5,Apartment For Rent,"$2,750/mo",2750.0,"1 bd • 1 ba • 496 sqft • 4,502 sqft lot • Built 1900",1; 1; 496; 4502; 1900,NaN,NaN,6.0,hours ago,NaN,NaN,1
12,http://www.zillow.com/homedetails/101-W-77th-St-APT-3C-New-York-NY-10024/2114511658_zpid/,4 photos,NaN,4.0,NaN,4 photos,4.0,http://www.zillow.com/homedetails/101-W-77th-St-APT-3C-New-York-NY-10024/2114511658_zpid/,"101 W 77th St APT 3C, New York, NY10024","101 W 77th St APT 3C, New York, NY Real Estate",101; 77; 3,Apartment For Rent,"$2,150/mo",2150.0,Studio • 1 ba,1,NaN,NaN,6.0,hours ago,NaN,NaN,1
13,http://www.zillow.com/homedetails/111-W-74th-St-APT-2R-New-York-NY-10023/2123304010_zpid/,6 photos,NaN,6.0,NaN,6 photos,6.0,http://www.zillow.com/homedetails/111-W-74th-St-APT-2R-New-York-NY-10023/2123304010_zpid/,"111 W 74th St APT 2R, New York, NY

In [16]:
# split using the bullet
def parse_info(row):
        if not 'sqft' in row:
            br, ba = row.split('•')[:2]
            sqft = np.nan
        else:
            br, ba, sqft = row.split('•')[:3]                
        return pd.Series({'Beds': br, 'Baths': ba, 'Sqft': sqft})

In [17]:
attr = sucln['propertyinfo_value'].applay(parse_info)
attr

AttributeError: 'Series' object has no attribute 'applay'

In [18]:
attr = sucln['propertyinfo_value'].apply(parse_info)
attr

,Baths,Beds,Sqft
2,1 ba,1 bd,NaN
3,2 ba,2 bds,"1,016 sqft"
4,1 ba,Studio,NaN
5,1 ba,2 bds,NaN
7,1 ba,2 bds,NaN
10,1 ba,1 bd,NaN
11,1 ba,1 bd,496 sqft
12,1 ba,Studio,NaN
13,1 ba,1 bd,NaN
17,1 ba,1 bd,NaN


In [19]:
# 在取值中将字符串删除
attr_cln = attr.applymap(lambda x: x.strip().split(' ')[0] if isinstance(x, str) else np.nan)

In [20]:
attr_cln

,Baths,Beds,Sqft
2,1,1,NaN
3,2,2,"1,016"
4,1,Studio,NaN
5,1,2,NaN
7,1,2,NaN
10,1,1,NaN
11,1,1,496
12,1,Studio,NaN
13,1,1,NaN
17,1,1,NaN


In [21]:
sujnd = sucln.join(attr_cln)
sujnd.T

,2,3,4,5,7,10,11,12,13,17,18,19,20,21,23,...,470,471,475,477,479,483,484,485,486,487,490,492,496,497,499
routablemask_link,http://www.zillow.com/homedetails/9-E-129th-St-1-New-York-NY-10035/2100761096_zpid/,http://www.zillow.com/homedetails/710-Riverside-Dr-APT-2C-New-York-NY-10031/124451755_zpid/,http://www.zillow.com/homedetails/413-E-84th-St-APT-8-New-York-NY-10028/2100761260_zpid/,http://www.zillow.com/homedetails/324-Pleasant-Ave-APT-6E-New-York-NY-10035/2100761258_zpid/,http://www.zillow.com/homedetails/217-Thompson-St-APT-6-New-York-NY-10012/2100761371_zpid/,http://www.zillow.com/homedetails/340-W-47th-St-APT-4C-New-York-NY-10036/2108861613_zpid/,http://www.zillow.com/homedetails/1356-1st-Ave-APT-5B-New-York-NY-10021/31539199_zpid/,http://www.zillow.com/homedetails/101-W-77th-St-APT-3C-New-York-NY-10024/2114511658_zpid/,http://www.zillow.com/homedetails/111-W-74th-St-APT-2R-New-York-NY-10023/2123304010_zpid/,http://www.zillow.com/homedetails/646-10th-Ave-APT-4B-New-York-NY-10036/2114397534_zpid/,http://www.zillow.com/homedetails/76-Carmine-St-APT-4C-New-York-NY-10014/2110813540_zpid/,http://www.zillow.com/homedetails/146-W-120th-St-APT-8-New-York-NY-10027/2100762986_zpid/,http://www.zillow.com/homedetails/137-W-72nd-St-APT-4FE-New-York-NY-10023/2105480982_zpid/,http://www.zillow.com/homedetails/178-Avenue-B-2-New-York-NY-10009/2100763265_zpid/,http://www.zillow.com/homedetails/95-Wall-St-72791-New-York-NY-10005/2103168440_zpid/,...,http://www.zillow.com/homedetails/304-E-41st-St-APT-1107A-New-York-NY-10017/2100774366_zpid/,http://www.zillow.com/homedetails/169-Avenue-A-APT-1-New-York-NY-10009/2105208117_zpid/,http://www.zillow.com/homedetails/722-10th-Ave-APT-3A-New-York-NY-10019/2126268388_zpid/,http://www.zillow.com/homedetails/200-W-70th-St-APT-11L3-New-York-NY-10023/2100774851_zpid/,http://www.zillow.com/homedetails/219-W-78th-St-APT-2F-New-York-NY-10024/2100775303_zpid/,http://www.zillow.com/homedetails/405-W-49th-St-APT-2W-New-York-NY-10019/2100776286_zpid/,http://www.zillow.com/homedetails/230-E-26th-St-APT-3A-New-York-NY-10010/2100776283_zpid/,http://www.zillow.com/homedetails/319-E-53rd-St-APT-2B-New-York-NY-10022/2104750345_zpid/,http://www.zillow.com/homedetails/53-Irving-Pl-APT-3K-New-York-NY-10003/2103128441_zpid/,http://www.zillow.com/homedetails/25-E-67th-St-APT-8D-New-York-NY-10065/2100777265_zpid/,http://www.zillow.com/homedetails/303-E-73rd-St-E-New-York-NY-10021/2100777278_zpid/,http://www.zillow.com/homedetails/422-E-81st-St-APT-4A-New-York-NY-10028/2111603077_zpid/,http://www.zillow.com/homedetails/445-W-49th-St-APT-3B-New-York-NY-10019/2106388119_zpid/,http://www.zillow.com/homedetails/203-E-121st-St-APT-901-New-York-NY-10035/2100777283_zpid/,http://www.zillow.com/homedetails/434-E-72nd-St-APT-2C-New-York-NY-10021/2120058527_zpid/
routablemask_link/_text,NaN,9 photos,5 photos,7 photos,NaN,NaN,9 photos,4 photos,6 photos,NaN,4 photos,6 photos,10 photos,6 photos,17 photos,...,5 photos,8 photos,5 photos,4 photos,6 photos,5 photos,6 photos,5 photos,6 photos,NaN,5 photos,4 photos,4 photos,4 photos,1 photo
routablemask_link/_title,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
routablemask_link_numbers,NaN,9,5,7,NaN,NaN,9,4,6,NaN,4,6,10,6,17,...,5,8,5,4,6,5,6,5,6,NaN,5,4,4,4,1
routablemask_content,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
imagebadge_value,NaN,9 photos,5 photos,7 photos,NaN,NaN,9 photos,4 photos,6 photos,NaN,4 photos,6 photos,10 photos,6 photos,17 photos,...,5 photos,8 photos,5 photos,4 photos,6 photos,5 photos,6 photos,5 photos,6 photos,NaN,5 photos,4 photos,4 photos,4 photos,1 photo
imagebadge_value_numbers,NaN,9,5,7,NaN,NaN,9,4,6,NaN,4,6,10,6,17,...,5,8,5,4,6,5,6,5,6,NaN,5,4,4,4,1
routable_link,http://www.zillow.com/homedetails/9-E-129th-St-1-New-York-NY-10035/2100761096_zpid/,http://www.zillow.com/homedetails/710-Riverside-Dr-APT